In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!nvidia-smi

Thu Aug 27 17:16:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateSchedule
from tensorflow.keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [3]:
(x_train,y_train), (x_test,y_test) = cifar100.load_data()

x_val, y_val = x_train[40000:], y_train[40000:]
x_train, y_train = x_train[:40000], y_train[:40000]

x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_val = x_val.reshape(x_val.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

y_train = keras.utils.to_categorical(y_train, 100)
y_val = keras.utils.to_categorical(y_val, 100)
y_test = keras.utils.to_categorical(y_test, 100)


pretrained_model = tf.keras.applications.ResNet50V2(include_top=False, 
                                                    weights='imagenet', 
                                                    input_tensor=None, 
                                                    input_shape=(128, 128, 3), 
                                                    classifier_activation='softmax')

model = Sequential([Input(shape=(32,32,3)),
                    UpSampling2D(),
                    UpSampling2D(),
                    pretrained_model,
                    GlobalAveragePooling2D(),
                    Dropout(0.5),
                    Dense(512, activation='relu'),
                    BatchNormalization(),
                    Dense(num_classes, activation='softmax')
                    ])
model.summary()

94674944/94668760 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 64, 64, 3)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 128, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 4, 4, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________

In [10]:
batch_size = 128
num_classes = 100
epochs = 90
lrate = 1e-3

In [4]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             horizontal_flip=True, 
                             fill_mode='nearest')
datagen.fit(x_train)
opt = keras.optimizers.Adam(learning_rate=lrate)

model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])

H = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), 
                        steps_per_epoch=len(x_train)/128,
                        validation_data=(x_val, y_val),
                        epochs=epochs)

Epoch 1/90
312/312 [==============================] - 73s 233ms/step - loss: 4.8272 - accuracy: 0.0121 - val_loss: 4.5780 - val_accuracy: 0.0161
Epoch 2/90
312/312 [==============================] - 71s 228ms/step - loss: 4.5619 - accuracy: 0.0248 - val_loss: 4.4943 - val_accuracy: 0.0447
Epoch 3/90
312/312 [==============================] - 71s 228ms/step - loss: 4.4811 - accuracy: 0.0439 - val_loss: 4.3804 - val_accuracy: 0.0690
Epoch 4/90
312/312 [==============================] - 71s 228ms/step - loss: 4.3860 - accuracy: 0.0618 - val_loss: 4.2552 - val_accuracy: 0.0916
Epoch 5/90
312/312 [==============================] - 71s 228ms/step - loss: 4.2692 - accuracy: 0.0816 - val_loss: 4.1030 - val_accuracy: 0.1210
Epoch 6/90
312/312 [==============================] - 71s 228ms/step - loss: 4.1392 - accuracy: 0.1052 - val_loss: 3.9275 - val_accuracy: 0.1505
Epoch 7/90
312/312 [==============================] - 71s 228ms/step - loss: 3.9786 - accuracy: 0.1346 - val_loss: 3.7582 - val_ac

In [11]:
score = model.evaluate(x_test, y_test)
print(f'Test loss: {score[0]:.4f}')
print(f'Test accuracy: {score[1]:.4f}')

313/313 [==============================] - 7s 24ms/step - loss: 0.8834 - accuracy: 0.7538
Test loss: 0.8834
Test accuracy: 0.7538


In [9]:
model.save('/content/drive/My Drive/model.h5')